# Data Loading

In [1]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import regularizers, optimizers
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
import re
from sklearn.decomposition import PCA
import keras
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from collections import Counter

In [15]:
len(tf.config.experimental.list_physical_devices("GPU"))

1

# Final Data

In [32]:
def list2str_fiksasidata():
    for i in range(len(df)):
        df['imageURLHighRes'][i] = 'gambar' + str(i) + '.jpg'
    
    return df.to_pickle('fixData.pickle') 
            
def unique_label():
    label = []
    for i in range(len(semiData2)):
        label.extend(semiData2['category'][i])
    
    return list(set(label))

In [ ]:
list2str_fiksasidata()

In [2]:
fixData = pd.read_pickle('fixData.pickle')

# Pemilihan Label

## Filter Noise

In [21]:
semiData = pd.read_pickle('dataSemiFinal.pickle')

In [ ]:
def filter_noise1():
    semiData['category'][98080].remove(None)
    for i in range(len(semiData)):
        semiData['category'][i] = [x for x in semiData['category'][i] if 
                                   len(list(x.split(" "))) <= 4]
    

filter_noise1()                        

In [ ]:
def filter_noise2():
    noise_copy = semiData['category']
    
    for i in range(len(noise_copy)):
        for y in semiData['category'][i]:
            x = y.split(" ")
            for j in x:
                a = re.findall('[0-9]+', j)
                if not a:
                    continue
                elif a[0].isdigit():
                    semiData['category'][i].remove(y)
                    
filter_noise2()

In [ ]:
semiData.to_pickle('dataSemiFinal2.pickle')

## Get Noise

In [ ]:
def get_noise():
    noise_copy = semiData['category']
    noise_collect = []
    
    for i in range(len(noise_copy)):
        for y in noise_copy[i]:
            x = y.split(" ")
            for j in x:
                a = re.findall('[0-9]+', j)
                if not a:
                    continue
                elif a[0].isdigit():
                    try:
                        noise_collect.append(y)
                    except ValueError:
                        continue
   
    for i in range(len(noise_copy)):
            noise_copy[i] = [x for x in noise_copy[i] if 
                             len(list(x.split(" "))) >= 5]

    label1 = []
    for i in range(len(noise_copy)):
        label1.extend(noise_copy[i])
        
    final_noise = list(set(noise_collect + label1))
    pd_noise = pd.DataFrame(final_noise, columns = ['Noise'])
    pd_noise.to_pickle('Noise.pickle')
    
get_noise()

## Inisialisasi Data Baru

In [10]:
semiData2 = pd.read_pickle('dataSemiFinal2.pickle')

In [2]:
noise = pd.read_pickle('Noise.pickle')
final_label = pd.read_pickle('labelFinal.pickle')

## Filter Noise (Manual)

In [3]:
noise_List = ['Bahan: Pemberian neoprene hipo-alergi',
              'Tidak beracun, perlindungan lingkungan','Bagus untuk pemberian hadiah!',
              'Fit klasik, Berat sedang','Bisa dicuci','Tidak ada',
              'Direkayasa untuk Menangkap Kelembaban', 'Dijamin mendapatkan banyak pujian!',
              'Perpipaan reflektif visibilitas tinggi.',
              'Memenuhi standar ANSI/ISEA','Retro, gaya antik', 'Kain tahan api',
              'Tidak ada jahitan bahu','Produk baru!',
              'Dibuat di Brasil!','WANITA','Asal: Diimpor',
              'A', 'Bahan berkualitas tinggi.', 'Sempurna untuk kondisi kerja',
              'Menggunakan Interlocking Tersembunyi',
              'Bahan: Aluminium Pesawat','Tahan Tusukan dan Potong', 'bisa dicuci dengan mesin',
              'E-Z Touch, Teknologi StayDark','kapas stapel ekstra panjang','Ringan dan nyaman dipakai',
              'Penggunaan yang Disarankan: bersepeda','pola merenda','dilepas','Tekan tombol',
              'Menangani Bahan: Stainless Steel','Tidak Disetujui TSA',
              'Cina','Gordini Usa','Dapat disesuaikan dan empuk','Bahan: campuran kapas',
              'rangka komposit','Bisa dicuci dengan mesin', 'Warna putih', 'Ini sangat deathrock,punk,goth',
              'Tiriskan Auger','Terpencil tersedia','Non-Polarisasi','Level','Kepala',
              'Bagian Depan Selesai Disesuaikan','Perhiasan Berkualitas Hebat!',
              'Bagus untuk penegakan hukum!','PENERBANGAN NAVY A.S. DISETUJUI',
              'NEW','Selangkangan Selangkangan','Atas: Kulit','polietilen','STRAPBACK UKURAN DEWASA, OSFM',
              'Tetapkan kombinasi Anda sendiri','Cangkang mikrofiber anti air','Bersihkan ujung jahitan buta.',
              'Dibuat di USA','AFCI tipe kombinasi','Gaya: Gaya Antik', 'Produk ini mudah dipasang',
              'pilihan tampilan', 'Ukuran: tampilkan sebagai gambar','tahan slip',
              'Penutupan: manset hook-and-loop','Panjang Poros Lainnya Tersedia',
              'Nama kategori: pisau','Ringan. Reflektif. FR.','orang berdosa',
              'Diproduksi di Cina','Gunakan untuk konstruksi baru','Penulis','Kain ringan memberikan kenyamanan',
              'Buatan tangan yang unik','Kulit full grain premium','Tiba di kotak hadiah',
              'Komponen baja berkualitas tinggi', 'Dilengkapi dengan dua kunci','Warna (s) - Hitam',
              'Dirakit di AS', 'Dibuat di Taiwan', 'Bernapas dan berinsulasi ringan',
              'Insole busa memori', 'saku ekstra dalam','(kosong)','Harga Untuk: Masing-masing',
              'Lengkap dengan tas penyimpanan','Sentuhan tangan yang lembut', 'tidak terpolarisasi',
              'LUXETJ','Konstruksi terpasang langsung','Mudah dipasang','Warna limau',
              'Disanitasi bertindak segar dirawat','dua kantong tempel besar','Penutupan: slip-on',
              'Bantalan: gel, Clarino','Tahan api cuaca panas','Tanpa jari','new','Midsole Tahan Tusukan',
              'Bahan: Kain Jaring','Tempat Bersih Saja','Termasuk penghapus kuku','Elemen pemanas',
              'Kulit Depan Tangan','Reflektif. Menyelamatkan Hidup. FRC.', 'Mata Bor yang Membosankan',
              'leher V',  'Tidak dapat diprogram','Gore-Tex','teknologi nano','Ratcheting',
              'Berlisensi Resmi', 'Tersedia dalam warna Hitam','Bahan','Sakelar yang Diaktifkan Gerakan',
              'Dukungan Rajutan Nilon', 'Tanda pangkat di bahu', 'Tahan abrasif','Cina Tulang',
              'Jalan Persegi', 'Tepi terikat poliester hitam.','Termasuk wadah penyimpanan neoprene',
              'Ukuran kecil untuk balita', 'Terdaftar UL', 'Belah kulit rusa','Shank nilon untuk penyangga',
              'Warna bersampul linen alami','Warna: Seperti Menampilkan Gambar', 'Selesai: Chrome yang dipoles',
              'Warna Cangkang: Putih','Pas Asli','Noda','FRC. Ringan. Terjangkau.','Sarung tangan ski wanita.',
              'Baju Kerja. FRC. Dasar.','Nyaman dan bernapas', 'kain-dan-sintetis','Mengapung',
              'Garansi seumur hidup','Menawarkan perlindungan debu maksimal','warna oranye',
              'Tempat Bersih','Bahan: Baja tahan karat','</span></span></span>', 'poliester',
              'Oranye & Putih','Membelitkan','Dijual di Amazon', 'Bunyi genta lonceng',
              'Tahan cuaca', 'Berkedip', 'Kualitas pelayanan tinggi','Aksesoris tidak termasuk', 
              'Sandal yang sangat nyaman','Seniman: Elena Julian','Dapat diprogram', 'Impor',
              'Selesaikan Senyawa','Kenyamanan dan ketangkasan ekstra',
              'Semuanya baja tahan karat','Tanpa tag','mencuci tangan',
              'BUATAN CHINA','Bisa Brankas', 'Sintetis','Dimensi: Lihat Gambar Spek','Mewah',
              'Menarik', 'bantalan', 'Keyed adalah kd','Dilengkapi dengan rantai','Transformer',
              'Buatan China', 'Cocok: Sesuai dengan Ukuran','Produk Berlisensi Resmi',
              'Bahan: PVC','Dukung tim favorit Anda','Satu saku depan','Menangani File',
              'Pas lurus','Bahan: Poliester dan Nilon', 'KUCING',
              'Merek baru dalam kotak!', 'Gaya: nakal','Pemasangan saluran','Kotak Hadiah Gratis',
              'busur derajat','Bahan: Paduan', 'Dapat Dicuci dengan Mesin', 'Bahan: Baja',
              'Penutupan ritsleting','Beralih', 'Termasuk: pisau gergaji','Multi-Outlet',
              'Pertolongan pertama pada kecelakaan','Jala poliester high-viz gaiter','Lensa Photochromic: tidak',
              'dua garis lengan','Peluang: Hotel, Bar, Rumah','Terpolarisasi','campuran poliester',
              'Fungsi tanggal','Garansi produsen',  'Bahan berkualitas',
              'Bahan: Katun, Akrilik','Bahan: Logam','Bahan: Plastik','Bahan: Tembaga, Kaca',
              'Baru','Berdiri','Besi tahan karat','Bisa dicuci. Gaya Slip-on','Cakupan: penuh',
              'Campuran, ,','Cangkang campuran aramid','Casing logam ringan','Cuci dengan mesin',
              'Cuci tangan','Cuci tangan saja','Dapat dibalik','Daya tahan dan ketangkasan',
              'Dekat Dengan Lampu Plafon', 'Dekat dengan Lampu Plafon','Dibuat Di Inggris', 
              'Dibuat dengan Elemen Swarovski','Dilengkapi dengan saku samping',
              'Dua garis vertikal','Finishing bodi merah','Finishing perunggu antik',
              'Gaya liar', 'Gaya luar biasa', 'Gaya: Mode','Hujan','Jahitan Ditempel.',
              'Kain Nilon, PU','Kap yang Dapat Dilepas',
              'Karbon poliamida anti-statis', 'Kelangsungan hidup kota','Lembut dan Lentur',
              'Lensa Terpolarisasi: tidak','Lensa polikarbonat','Lensa: Lexan',
              'Material buatan manusia','Matrial: Paduan','Memangkas','Merek: PenSee',
              'Pemrograman tujuh hari','Putih hitam','RUBBERMAID COMMERCIAL PROD.',
              'bahaya listrik','kayu ek berkualitas tinggi','gabungan','penutupan tali',
              'tali yang dapat disesuaikan','terisolasi', 'terpolarisasi']

noise_List2 = ['Bilah telapak tangan khusus','Bingkai asetat','Bingkai plastik',
               'Bingkai: uretan','Buatan tangan di Nepal','Desain busana.','Desain ergonomis',
               'Desain indoor-outdoor', 'Detail tombol logo epoksi','Di kantong permen bir.',
               'Dukungan Ergonomis', 'Dukungan Kembali', 'Fluks mematri','Gembok berkunci',
               'Gulung penuh','Gunting pembuka botol','Hitam pekat',
               'Kain politeks tahan air','Konstruksi Kulit Rusa Penuh', 'Konstruksi logam',
               'Konstruksi membran tahan air', 'Konstruksi wesel Goodyear','Kristal Austria',
               'Kulit hitam','Membuat hadiah yang bagus!', 'Memperbaiki Pancuran',
               'Menangkap','Menarik Pegangan','Mengetuk','Menghiasi Kuku','Mengikat Kulit Berbalik',
               'Oranye','Nosepiece karet fleksibel',
               'Renda oxford lima lubang','Sarung tangan sekali pakai','Sarung tangan tahan panas',
               'Satu ukuran paling cocok','Shell dengan lapisan cetak.','Tahan Air dan Minyak',
               'Tahan air','Tahan air dan bernapas.','Tahan air.','Tahan angin.','Tahan guncangan',
               'Tahan lama','Tahan pecah','Tahan pudar','Tahan selip','bingkai plastik','iridium',
               'kertas tahan lama','Warna hitam','Warna: Hitam dan Abu-abu','Antimikroba',
               'Hem Manset Kulit Digulung','Hemat energi','Kenyamanan dan Ketangkasan',
               'Kuil fleksibel punggung lurus','Lap kain terry bawaan.','Lapangan Pertukangan',
               'Lapisan Aramid Campuran','Lapisan bawah','Lapisan poliester','Lensa plastik',
               'Penguasa','Poliamida, ,','Poliester','Poliuretan, Lak & Pernis', 'Pria',
               'Produk Baru','Produk baru.','Warna hitam','Warna: Hitam dan Abu-abu'
              ]


In [5]:
noise_List3 = noise['Noise'].values.tolist()
final_noise = noise_List + noise_List2 + noise_List3

# PCA

In [ ]:
def reduksi_dim():
    pca = PCA(n_components = 1)
    pca.fit(final_label[final_noise])
    final_label['Noise'] = pca.transform(final_label.loc[:, tuple(final_noise)]).flatten()
    final_label.drop(final_noise, axis = 1, inplace = True)
    final_label['Noise'].to_pickle('noisePCA.pickle')

reduksi_dim()

# Generalisasi Label Tertentu

In [ ]:
lab_jenis = ['Adaptor', 'Aksesori', 'Bathtub', 'Brankas', 'Cat', 'Faucet', 'Gergaji', 'Gulungan',
             'Kabel', 'Jendela','Kit', 'Klem', 'Konektor', 'Kunci', 'Lampu', 'Lantai',
             'Motor', 'Obeng', 'Obor','Pahat','Toilet', 'Dispenser', 
             'Gembok', 'Kacamata', 'Gunting', 'Laser',
             'Palu', 'Pemaku', 'Pemotong',  'Penggiling','Penguji','Pengukur',
             'Peralatan', 'Perlindungan','Pintu', 'Pisau', 'Pompa', 'Pukulan',
             'Rak','Sakelar', 'Saluran', 'Sander', 'Sekering', 'Selang',
             'Senjata', 'Senter', 'Set', 'Soket', 'Sol', 'Stapler','Suede',  'Tali',
             'Tang','Tangga','Topi','Ubin', 'Ventilasi','Wastafel', 'lensa',
             'Bit', 'Filter', 'Katup', 'Kompresor', 'Pencahayaan', 'Kulit'
            ]

lab_jenis2 = ['Kotak Surat', 'Mata Bor', 'Mata Gergaji', 'Mesin Bor', 'Mesin Bubut',
              'Pemutus Sirkuit', 'Sarung Tangan', 'Suku Cadang']

#'Sekring', #'Braket', Lampu: L.E.D. Bertenaga Tinggi, 'Mata bor', Sarung tangan
#Kotak Surat  #'Mata Bor' #'Mata Gergaji' #Mesin Bor #Mesin Bubut
#Pemutus Sirkuit #Moulding & Trim' #'Sarung Tangan' #'Suku Cadang


def gen_label(text):
    copy = semiData2['category']
    
    for i in range(len(copy)):
        for y in copy[i]:
            if y == text:
                continue
            else:
                x = y.split(" ")
                if not x or len(x) < 2: #Kondisi sesuai kebutuhan 
                    continue
                elif x[0] + ' ' + x[1] == text: ##Kondisi sesuai kebutuhan
                    try:
                        semiData2['category'][i].append(text)
                        semiData2['category'][i].remove(y)
                    except ValueError:
                        continue
    
for text in lab_pilah2:
    gen_label(text)

def gen_label_sama(text):
    copy = semiData2['category']
    
    for i in range(len(copy)):
        for y in copy[i]:
            if y == text:
                semiData2['category'][i].append(text)
                semiData2['category'][i].remove(y)
                
gen_label_sama(text)

# Finalisasi Data 

# Multi-Label Encoding

In [24]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
def label_image():
    label = [tuple(semiData2['category'][i]) for i in range(len(semiData2))]
    label_final = pd.DataFrame(mlb.fit_transform(label), 
                               columns=mlb.classes_)
    label_final.to_pickle('labelFinal2.pickle')
    #Terdapat 2 data yang disimpan
    
label_image()

TypeError: '<' not supported between instances of 'str' and 'NoneType'

# Finalisasi Data 

In [ ]:
label_siap = pd.read_pickle('labelFinal2.pickle')
label_siap = label_siap.drop(drop_list + drop_List2, axis = 1)

noisePCA = pd.read_pickle('noisePCA.pickle')

label_siapsekali = pd.concat([label_siap, noisePCA], axis=1, join='inner')
label_siapsekali.to_pickle('labelSiap.pickle')

labelSiap = pd.read_pickle('labelSiap.pickle')
new_data = label_siapsekali = pd.concat([fixData['imageURLHighRes'], labelSiap], axis=1, join='inner')
new_data.to_pickle('final_DataFrame.pickle')

In [2]:
df_final = pd.read_pickle('finalDataFrame2.pickle')

In [3]:
for i in range(len(df_final)):
    if df_final['Noise'][i] > df_final['Noise'].mean():
        df_final['Noise'][i] = 1
    else: 
        df_final['Noise'][i] = 0

C:\Users\dimas\AppData\Local\Temp\ipykernel_20560\236096813.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Noise'][i] = 0
C:\Users\dimas\AppData\Local\Temp\ipykernel_20560\236096813.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Noise'][i] = 1


In [5]:
df_final['Noise'] = df_final['Noise'].astype(int)

In [10]:
count = 0

for i in range(len(df_final['Noise'])):
    if df_final['Noise'][i] == 1:
        count=count+1

In [11]:
count

33192

## Pembobotan Pada Label

In [26]:
labels_count = Counter(label for lbs in semiData2['category'] for label in lbs)
labels_count_drop = [label for label in list(labels_count.keys()) if labels_count[label] < 100]
labels_count_drop.append('Alat & Perbaikan Rumah')

In [ ]:
wow = ['Berdiri', 'Level', 'Dukungan Ergonomis', 'Lapangan Pertukangan', 'busur derajat', 
       'Multi-Outlet', 'Mengetuk', 'Ratcheting', 'Membelitkan', 'Penguasa', 'Menangkap', 
       'Tahan cuaca', 'Bunyi genta lonceng', 'Noda', 'Dapat diprogram', 'Memangkas', 'Kepala', 'Tiriskan Auger', 
       'Moulding & Trim', 'Poliuretan, Lak & Pernis', 'Suku Cadang', 'Tekan tombol', 'Impor']

for i in drop_label: 
    labels_count_drop.remove(i)

In [44]:
label_drop1 = ['Bahan', 'Mengapung', 'Penulis', 'orang berdosa', 'Menggunakan Interlocking Tersembunyi', 
               'Menangani File', 'Jalan Persegi', 'Menarik Pegangan', 'Belah kulit rusa', 
               'Pertolongan pertama pada kecelakaan', 'Berkedip', 'Material buatan manusia',
               'Lapisan Aramid Campuran', 'Dukungan Rajutan Nilon', 'Lembut dan Lentur', 
               'Bisa dicuci dengan mesin', 'Sempurna untuk kondisi kerja', 'Desain ergonomis', 
               'Fluks mematri', 'Garansi seumur hidup', 'Tidak dapat diprogram', 'Dukungan Kembali',
               'Pemasangan saluran', 'Selesaikan Senyawa', 'Nosepiece karet fleksibel', 'Produk Baru',
               'Oranye', 'Kuil fleksibel punggung lurus', 'Termasuk wadah penyimpanan neoprene', 
               'Bisa dicuci. Gaya Slip-on', '(kosong)', 'Poliester', 'Warna (s) - Hitam', 
               'PENERBANGAN NAVY A.S. DISETUJUI', 'Ukuran kecil untuk balita', 'campuran poliester',
               'Dilengkapi dengan saku samping', 'Produk baru.', 'Transformer', 'Besi tahan karat',
               'Kelangsungan hidup kota', 'Nama kategori: pisau', '</span></span></span>', 
               'Tahan Tusukan dan Potong', 'Kenyamanan dan Ketangkasan', 'Cangkang campuran aramid', 
               'Kenyamanan dan ketangkasan ekstra', 'Lapisan bawah', 'polietilen', 'saku ekstra dalam', 
               'dua kantong tempel besar', 'bingkai plastik', 'tidak terpolarisasi', 'Kain politeks tahan air',
               'Pria', 'Selangkangan Selangkangan', 'E-Z Touch, Teknologi StayDark', 'Tahan api cuaca panas',
               'Antimikroba', 'Memenuhi standar ANSI/ISEA', 'Pemrograman tujuh hari', 'Mudah digunakan',
               'pilihan tampilan', 'Pas Asli', 'Tidak ada jahitan bahu', 'terpolarisasi', 
               'Kotak Hadiah Gratis', 'Direkayasa untuk Menangkap Kelembaban', 'Insole busa memori', 
               'Aksesoris tidak termasuk', 'Bersihkan ujung jahitan buta.', 'Terpolarisasi',
               'Bagian Depan Selesai Disesuaikan', 'Lap kain terry bawaan.', 'Cina', 
               'Panjang Poros Lainnya Tersedia', 'AFCI tipe kombinasi', 'Terdaftar UL', 
               'Konstruksi wesel Goodyear', 'tali yang dapat disesuaikan', 'Gaya luar biasa',
               'Garansi produsen', 'Elemen pemanas', 'Cuci tangan', 'Cangkang mikrofiber anti air',
               'Nyaman dan bernapas', 'Manset elastis dengan hook-and-loop', 'Pas lurus',
               'Sentuhan tangan yang lembut', 'Harga Untuk: Masing-masing', 'Bingkai asetat',
               'Lensa polikarbonat', 'Non-Polarisasi', 'Fungsi tanggal', 'Di kantong permen bir.', 
               'Tiba di kotak hadiah', 'Dibuat dengan Elemen Swarovski', 'Produk Berlisensi Resmi',
               'Disanitasi bertindak segar dirawat', 'Tanpa tag', 'Putih hitam', 'Penutupan ritsleting',
               'Midsole Tahan Tusukan', 'Bisa dicuci', 'Penutupan: manset hook-and-loop', 'Desain busana.',
               'Semuanya baja tahan karat', 'Bahan berkualitas', 'Baru', 'Tahan air', 'WANITA',
               'Tempat Bersih Saja', 'Tahan guncangan', 'Warna hitam', 'Bagus untuk pemberian hadiah!',
               'warna oranye', 'Jala poliester high-viz gaiter', 'Warna limau', 
               'Bahan: Paduan', 'Warna: Seperti Menampilkan Gambar', 'Buatan China',
               'Gaya: Mode', 'Kristal Austria', 'Tahan pudar', 'Bantalan: gel, Clarino', 
               'Penutupan: slip-on', 'LUXETJ', 'rangka komposit', 'Karbon poliamida anti-statis', 
               'Daya tahan dan ketangkasan', 'Mewah', 'Gore-Tex', 'Bingkai plastik', 'Lensa plastik',
               'Warna bersampul linen alami', 'Tetapkan kombinasi Anda sendiri', 'Tidak Disetujui TSA',
               'Dapat Dicuci dengan Mesin', 'Cina Tulang', 'Dibuat di USA', 'Tanpa jari', 'poliester',
               'Dimensi: Lihat Gambar Spek', 'Dapat disesuaikan dan empuk', 'Berlisensi Resmi', 'A', 
               'Bahan: Logam', 'Dilengkapi dengan dua kunci', 'Keyed adalah kd', 'Finishing bodi merah',
               'Dibuat di Taiwan', 'Retro, gaya antik', 'Finishing perunggu antik', 'Dilengkapi dengan rantai', 
               'Matrial: Paduan', 'Dukung tim favorit Anda', 'Hitam pekat', 'Oranye & Putih', 'Menghiasi Kuku', 
               'Produk ini mudah dipasang', 'Diproduksi di Cina', 'Menawarkan perlindungan debu maksimal', 
               'Bagus untuk penegakan hukum!', 'Dibuat di Brasil!', 'Perhiasan Berkualitas Hebat!', 
               'Dijual di Amazon', 'Merek: PenSee', 'Konstruksi terpasang langsung', 'Shank nilon untuk penyangga',
               'Satu saku depan', 'Seniman: Elena Julian', 'Menangani Bahan: Stainless Steel', 'Gordini Usa', 
               'NEW', 'bantalan', 'terisolasi', 'Asal: Diimpor', 'Cocok: Sesuai dengan Ukuran', 'Cakupan: penuh',
               'Gaya: nakal', 'Bilah telapak tangan khusus', 'Hemat energi', 'Kualitas pelayanan tinggi', 
               'Tahan Air dan Minyak', 'Tahan abrasif', 'Tahan lama', 'Tepi terikat poliester hitam.', 
               'STRAPBACK UKURAN DEWASA, OSFM', 'BUATAN CHINA', 'Campuran, ,', 
               'Perpipaan reflektif visibilitas tinggi.', 'Sintetis', 'KUCING', 
               'Mudah dipasang', 'kayu ek berkualitas tinggi', 'Komponen baja berkualitas tinggi', 
               'bisa dicuci dengan mesin', 'Bahan: PVC', 'Tidak beracun, perlindungan lingkungan', 'dilepas', 
               'Dua garis vertikal', 'dua garis lengan', 'Gaya liar', 'Dibuat Di Inggris', 'Terpencil tersedia',
               'bahaya listrik', 'gabungan', 'Gunakan untuk konstruksi baru', 'Bingkai: uretan', 'Lensa: Lexan',
               'Lensa Terpolarisasi: tidak', 'Lensa Photochromic: tidak', 'Sandal yang sangat nyaman',
               'Tersedia dalam warna Hitam', 'Desain indoor-outdoor', 'Tahan selip', 'Produk baru!', 
               'Penggunaan yang Disarankan: bersepeda', 'Kain ringan memberikan kenyamanan', 
               'Kap yang Dapat Dilepas', 'Satu ukuran paling cocok', 'Kain Nilon, PU', 'Tidak ada', 
               'Konstruksi logam', 'leher V', 'Fit klasik, Berat sedang', 'iridium', 'Lapisan poliester', 
               'Poliamida, ,', 'Tahan air dan bernapas.', 'Tanda pangkat di bahu', 'Detail tombol logo epoksi',
               'Bernapas dan berinsulasi ringan', 'Gulung penuh', 'Dapat dibalik', 'Dirakit di AS', 'Bahan: Baja',
               'Gaya: Gaya Antik', 'Peluang: Hotel, Bar, Rumah', 'Lengkap dengan tas penyimpanan', 'Bahan: Kain Jaring', 
               'Ukuran: tampilkan sebagai gambar', 'tahan slip', 'Bahan: Tembaga, Kaca', 'Selesai: Chrome yang dipoles', 'Ringan dan nyaman dipakai', 'Buatan tangan yang unik', 'Warna Cangkang: Putih', 'Bahan berkualitas tinggi.', 'Merek baru dalam kotak!', 'Dijamin mendapatkan banyak pujian!', 'Membuat hadiah yang bagus!', 'Buatan tangan di Nepal', 'pola merenda', 'kapas stapel ekstra panjang', 'teknologi nano', 'Bahan: campuran kapas', 'Ini sangat deathrock,punk,goth', 'Cuci tangan saja', 'Bahan: Baja tahan karat', 'Renda oxford lima lubang', 'kain-dan-sintetis', 'Bahan: Plastik', 'Shell dengan lapisan cetak.', 'Tahan air.', 'Tahan angin.', 'Jahitan Ditempel.', 'Konstruksi membran tahan air', 'Bahan: Poliester dan Nilon', 'Warna: Hitam dan Abu-abu', 'Bahan: Pemberian neoprene hipo-alergi', 'mencuci tangan', 'Cuci dengan mesin', 'Bahan: Aluminium Pesawat', 'penutupan tali', 'Reflektif. Menyelamatkan Hidup. FRC.', 'Kain tahan api', 'FRC. Ringan. Terjangkau.', 'Ringan. Reflektif. FR.', 'Baju Kerja. FRC. Dasar.', 'Tempat Bersih', 'Bahan: Katun, Akrilik', 'new', 'RUBBERMAID COMMERCIAL PROD.', 'Warna putih', 'Termasuk penghapus kuku', 'Termasuk: pisau gergaji', 'Casing logam ringan', 'Tahan pecah', 'kertas tahan lama']

label_drop2 = ['Mengapung', 'orang berdosa', 'Menangani File', 'Menarik Pegangan',
               'Pertolongan pertama pada kecelakaan', 'Material buatan manusia', 
               'Dukungan Rajutan Nilon', 'Bisa dicuci dengan mesin', 'Desain ergonomis',
               'Garansi seumur hidup', 'Dukungan Kembali', 'Selesaikan Senyawa', 'Produk Baru',
               'Kuil fleksibel punggung lurus', 'Bisa dicuci. Gaya Slip-on', 'Poliester',
               'PENERBANGAN NAVY A.S. DISETUJUI', 'campuran poliester', 'Produk baru.', 'Besi tahan karat', 
               'Nama kategori: pisau', 'Tahan Tusukan dan Potong', 'Cangkang campuran aramid', 'Lapisan bawah', 
               'saku ekstra dalam', 'bingkai plastik', 'Kain politeks tahan air', 'Selangkangan Selangkangan',
               'Tahan api cuaca panas', 'Memenuhi standar ANSI/ISEA', 'Mudah digunakan', 'Pas Asli', 'terpolarisasi', 
               'Direkayasa untuk Menangkap Kelembaban', 'Aksesoris tidak termasuk', 'Terpolarisasi', 
               'Lap kain terry bawaan.', 'Panjang Poros Lainnya Tersedia', 'Terdaftar UL', 
               'tali yang dapat disesuaikan', 'Garansi produsen', 'Cuci tangan', 'Nyaman dan bernapas',
               'Pas lurus', 'Harga Untuk: Masing-masing', 'Lensa polikarbonat', 'Fungsi tanggal',
               'Tiba di kotak hadiah', 'Produk Berlisensi Resmi', 'Tanpa tag', 'Penutupan ritsleting',
               'Bisa dicuci', 'Desain busana.', 'Bahan berkualitas', 'Tahan air', 'Tempat Bersih Saja',
               'Warna hitam', 'warna oranye', 'Warna limau', 'Warna: Seperti Menampilkan Gambar', 
               'Gaya: Mode', 'Tahan pudar', 'Penutupan: slip-on', 'rangka komposit', 'Daya tahan dan ketangkasan', 
               'Gore-Tex', 'Lensa plastik', 'Tetapkan kombinasi Anda sendiri', 'Dapat Dicuci dengan Mesin', 
               'Dibuat di USA', 'poliester', 'Dapat disesuaikan dan empuk', 'A', 'Dilengkapi dengan dua kunci',
               'Finishing bodi merah', 'Retro, gaya antik', 'Dilengkapi dengan rantai', 'Dukung tim favorit Anda', 
               'Oranye & Putih', 'Produk ini mudah dipasang', 'Menawarkan perlindungan debu maksimal', 'Dibuat di Brasil!', 
               'Dijual di Amazon', 'Konstruksi terpasang langsung', 'Satu saku depan', 'Menangani Bahan: Stainless Steel',
               'NEW', 'terisolasi', 'Cocok: Sesuai dengan Ukuran', 'Gaya: nakal', 'Hemat energi', 
               'Tahan Air dan Minyak', 'Tahan lama', 'STRAPBACK UKURAN DEWASA, OSFM', 'Campuran, ,',
               'Sintetis', 'Mudah dipasang', 'Komponen baja berkualitas tinggi', 'Bahan: PVC', 'dilepas',
               'dua garis lengan', 'Dibuat Di Inggris', 'bahaya listrik', 'Gunakan untuk konstruksi baru', 
               'Lensa: Lexan', 'Lensa Photochromic: tidak', 'Tersedia dalam warna Hitam', 'Tahan selip', 
               'Penggunaan yang Disarankan: bersepeda', 'Kap yang Dapat Dilepas', 'Kain Nilon, PU', 'Konstruksi logam', 
               'Fit klasik, Berat sedang', 'Lapisan poliester', 'Tahan air dan bernapas.', 'Detail tombol logo epoksi',
               'Gulung penuh', 'Dirakit di AS', 'Gaya: Gaya Antik', 'Lengkap dengan tas penyimpanan',
               'Ukuran: tampilkan sebagai gambar', 'Bahan: Tembaga, Kaca', 'Ringan dan nyaman dipakai',
               'Warna Cangkang: Putih', 'Merek baru dalam kotak!', 'Membuat hadiah yang bagus!', 'pola merenda',
               'teknologi nano', 'Ini sangat deathrock,punk,goth', 'Bahan: Baja tahan karat', 'kain-dan-sintetis',
               'Shell dengan lapisan cetak.', 'Tahan angin.', 'Konstruksi membran tahan air', 'Warna: Hitam dan Abu-abu', 
               'mencuci tangan', 'Bahan: Aluminium Pesawat', 'Reflektif. Menyelamatkan Hidup. FRC.', 'FRC. Ringan. Terjangkau.',
               'Baju Kerja. FRC. Dasar.', 'Bahan: Katun, Akrilik', 'RUBBERMAID COMMERCIAL PROD.', 'Termasuk penghapus kuku',
               'Casing logam ringan', 'kertas tahan lama']

label_drop = label_drop1+label_drop2

#for i in label_drop: 
#    labels_count_drop.remove(i)

In [37]:
df_final = df_final.drop(labels_count_drop, axis = 1)

KeyError: "['Belt Sanders', 'Lembar Sander', 'Pengupas & Penghilang Cat', 'Penyambung Biskuit & Piring', 'Mesin Bor', 'Pangkalan Seluler', 'Alat berosilasi', 'Tabel', 'Mortiser', 'Perancah & Aksesoris', 'Lift Drywall', 'Selesaikan Stapler', 'Pemegang Bit', 'Bor Sudut Kanan', 'Celemek', 'Penutup Rol', 'Kartrid', 'Flapper Toilet', 'Penarik Kuku', 'Pemadam api', 'Paint Edger', 'Rangka Rol Cat', 'Ember Cat', 'Bar Grounding', 'Hub', 'Remot kontrol', 'Perlengkapan Binatu & Utilitas', 'Pesawat Tangan', 'Roda Pemotong', 'Bahan', 'Kain Perangkat Keras', 'Lubang Saw Arbors', 'Tas', 'Nibbler', 'Finish Nailer', 'Mengapung', 'Penulis', 'mesin bubut', 'Mesin Bubut', 'pesawat layang', 'Pembuangan & Bagian Sampah', 'Pembuangan Sampah', 'Pasang Sekering', 'Bilah Gergaji Gulir', 'Pengamplasan Disk', 'Kerek', 'Garis Kapur', 'Planer Genggam', 'Piala Furnitur', 'Tiang Ekstensi Rol Cat', 'Pegas & Kabel', 'Gagang Palu', 'Detil Sanders', 'Pengupasan Cuaca', 'Segel Pintu Garasi', 'Roda Pengukur', 'Penghilang S

In [3]:
classes = list(df_final.columns)
classes.remove('imageURLHighRes')

In [3]:
df_final

,imageURLHighRes,Adaptor,Aerator,Aerator Wastafel Dapur,Aksesori,Alas Kaki Keselamatan,Alat & Perangkat Keras,Alat Batu,Alat Listrik,Alat Penanda,...,Truk Tangan,Tuas Pintu,Ubin,Vakum Basah-Kering,Veneer,Ventilasi,Wallpaper,Wastafel,Windows,Noise
0,gambar0.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,gambar1.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,gambar2.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,gambar3.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,gambar4.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273533,gambar273533.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
273534,gambar273534.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
273535,gambar273535.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
273536,gambar273536.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
def label_dict():
    label = []
    count_label = []
    for i in range(len(classes)):
        label.append(i)
        count_label.append(labels_count[classes[i]])
    
    return dict(zip(label, count_label))
        
new_labelbobot = label_dict()

In [40]:
new_labelbobot[325] = 273538*10

In [41]:
class_weights = {cls: len(df_final) / count for cls, count in new_labelbobot.items()}

In [42]:
import random

for i in range(len(classes)):
    if class_weights[i] > 10:
        class_weights[i] = random.uniform(3, 5)

In [43]:
class_weights

{0: 4.046902128139208,
 1: 3.1074019281291347,
 2: 3.6724340849803156,
 3: 4.875341627482433,
 4: 3.6023960104101365,
 5: 3.0487130172752868,
 6: 3.06601984191549,
 7: 3.924534711321447,
 8: 3.8298224476453653,
 9: 4.381936213639008,
 10: 3.628373509312455,
 11: 3.515316407676072,
 12: 3.883666845692856,
 13: 3.946554435469685,
 14: 3.6976636726450534,
 15: 3.9879086035476634,
 16: 3.6149766486180015,
 17: 4.282154088905256,
 18: 3.2865328770431574,
 19: 4.324060671048077,
 20: 4.057025724085422,
 21: 4.432257704054482,
 22: 3.865074951847154,
 23: 3.0670962943193514,
 24: 3.2276437439612695,
 25: 3.105813784125411,
 26: 4.516894110440882,
 27: 4.730888222873634,
 28: 4.9400512965589805,
 29: 4.500230508774069,
 30: 3.6267926965484873,
 31: 3.549520717998118,
 32: 4.753616242795514,
 33: 3.5911023187291717,
 34: 4.568905231343507,
 35: 3.835987388118954,
 36: 4.932557807917469,
 37: 3.1470581241237454,
 38: 3.1802858054772742,
 39: 3.554028108650563,
 40: 4.92035437581564,
 41: 4.36451

In [44]:
new_labelbobot

{0: 926,
 1: 102,
 2: 115,
 3: 7342,
 4: 456,
 5: 184,
 6: 302,
 7: 6955,
 8: 162,
 9: 411,
 10: 11036,
 11: 294,
 12: 254,
 13: 1311,
 14: 435,
 15: 398,
 16: 111,
 17: 424,
 18: 350,
 19: 2207,
 20: 650,
 21: 526,
 22: 476,
 23: 578,
 24: 6509,
 25: 140,
 26: 106,
 27: 472,
 28: 239,
 29: 558,
 30: 173,
 31: 151,
 32: 129,
 33: 443,
 34: 100,
 35: 240,
 36: 121,
 37: 2264,
 38: 17550,
 39: 147,
 40: 322,
 41: 759,
 42: 1200,
 43: 115,
 44: 1838,
 45: 487,
 46: 487,
 47: 291,
 48: 101,
 49: 510,
 50: 465,
 51: 2031,
 52: 458,
 53: 479,
 54: 122,
 55: 522,
 56: 195,
 57: 313,
 58: 1161,
 59: 126,
 60: 9026,
 61: 232,
 62: 2028,
 63: 451,
 64: 1340,
 65: 2223,
 66: 452,
 67: 2494,
 68: 163,
 69: 378,
 70: 5316,
 71: 107,
 72: 373,
 73: 315,
 74: 163,
 75: 102,
 76: 1276,
 77: 1472,
 78: 1766,
 79: 276,
 80: 291,
 81: 115,
 82: 960,
 83: 400,
 84: 196,
 85: 1381,
 86: 1764,
 87: 316,
 88: 545,
 89: 572,
 90: 23952,
 91: 2697,
 92: 1237,
 93: 325,
 94: 168,
 95: 402,
 96: 1236,
 97: 161,


# Transformasi Data

In [ ]:
mlb = MultiLabelBinarizer()
def label_image():
    label = [tuple(semiData2['category'][i]) for i in range(len(semiData2))]
    label_final = pd.DataFrame(mlb.fit_transform(label), 
                               columns=mlb.classes_)
    label_final.to_pickle('labelFinal2.pickle')
    
label_image()

In [10]:
def data_trans():
    datagen=ImageDataGenerator(rescale=1./255.)
    
    test_datagen=ImageDataGenerator(rescale=1./255.)
    
    train_generator=datagen.flow_from_dataframe(
            dataframe=df_final[:218830],
            directory="./gambar",
            x_col="imageURLHighRes",
            y_col= classes,
            batch_size = 32,
            seed=42,
            shuffle=True,
            class_mode="raw",
            target_size=(224,224))
    
    valid_generator=test_datagen.flow_from_dataframe(
            dataframe=df_final[218830:246184],
            directory="./gambar",
            x_col="imageURLHighRes",
            y_col= classes,
            batch_size = 32,
            seed=42,
            shuffle=True,
            class_mode="raw",
            target_size=(224,224))
    
    test_generator=test_datagen.flow_from_dataframe(
            dataframe=df_final[246184:],
            directory="./gambar",
            x_col="imageURLHighRes",
            batch_size=1,
            seed=42,
            shuffle=False,
            class_mode=None,
            target_size=(224,224))
    
    return train_generator, valid_generator, test_generator

# Model CNN

In [12]:
def fir_savemodel():
    train_generator, valid_generator, _ = data_trans()
    
    STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
    STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
    #STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
    
    #Model CNN
    model = Sequential()
    model.add(Conv2D(32, (3, 3),
                     input_shape=(100,100,3)))
    model.add(Activation(tf.keras.layers.LeakyReLU()))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation(tf.keras.layers.LeakyReLU()))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(128))
    model.add(Dense(326, activation='sigmoid'))
    
    model.compile(optimizer = tf.keras.optimizers.Adam(1e-5),
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=["accuracy",
                           keras.metrics.TruePositives(name='tp'),
                           keras.metrics.FalsePositives(name='fp'),
                           keras.metrics.TrueNegatives(name='tn'),
                           keras.metrics.FalseNegatives(name='fn'),   
                           keras.metrics.Precision(name='precision'),
                           keras.metrics.Recall(name='recall'),
                           keras.metrics.AUC(name='auc'),
                          ])
    
    filepath = 'training_3/cp--{epoch:02d}.ckpt'
    checkpoint = keras.callbacks.ModelCheckpoint(filepath=filepath, period=5)
    
    model.fit_generator(epochs = 10, generator=train_generator,
                        steps_per_epoch=STEP_SIZE_TRAIN,
                        validation_data=valid_generator,
                        validation_steps=STEP_SIZE_VALID,
                        callbacks = [checkpoint]
                        ,class_weight=class_weights
    )

In [ ]:
fir_savemodel()

Found 218830 validated image filenames.
Found 27354 validated image filenames.
Found 27354 validated image filenames.


C:\Users\dimas\AppData\Local\Temp\ipykernel_7224\1382829167.py:38: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(epochs = 10, generator=train_generator,


Epoch 1/10
6838/6838 [==============================] - 1666s 243ms/step - loss: 0.1909 - accuracy: 0.0535 - tp: 4029.0000 - fp: 368596.0000 - tn: 70428848.0000 - fn: 526669.0000 - precision: 0.0108 - recall: 0.0076 - auc: 0.8207 - val_loss: 0.0313 - val_accuracy: 0.0839 - val_tp: 68.0000 - val_fp: 408.0000 - val_tn: 8844211.0000 - val_fn: 64241.0000 - val_precision: 0.1429 - val_recall: 0.0011 - val_auc: 0.8657
Epoch 2/10
4062/6838 [================>.............] - ETA: 7:31 - loss: 0.1280 - accuracy: 0.0624 - tp: 696.0000 - fp: 962.0000 - tn: 42058716.0000 - fn: 314458.0000 - precision: 0.4198 - recall: 0.0022 - auc: 0.8639

In [ ]:
def checkp_train():
    train_generator, valid_generator, _ = data_trans()

    STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
    STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

    checkpoint = keras.callbacks.ModelCheckpoint("training_2/cp--{epoch:02d}.ckp", period= 5)

    loaded_model = keras.models.load_model('training_1/cp--05.ckp')
    new_history = loaded_model.fit_generator(epochs = 10, 
                                             generator=train_generator,
                                             steps_per_epoch=STEP_SIZE_TRAIN,
                                             validation_data=valid_generator,
                                             validation_steps=STEP_SIZE_VALID,
                                             callbacks = [checkpoint],
                                             class_weight=class_weights)

checkp_train()